## LOAD SOLUTION FILES

In [2]:
import os
import pickle
import pandas as pd
from utils import * 
from attention import *
from helpers import *

In [255]:
from torch.autograd import Variable

def nopeak_mask(size):
    """Tạo mask được sử dụng trong decoder để lúc dự đoán trong quá trình huấn luyện
     mô hình không nhìn thấy được các từ ở tương lai
    """
    np_mask = np.triu(np.ones((1, size, size)), k=1).astype('uint8')
    np_mask =  Variable(torch.from_numpy(np_mask) == 0)
    # np_mask = np_mask.to(device)

    return np_mask

def create_masks(src, trg):
    """ Tạo mask cho encoder,
    để mô hình không bỏ qua thông tin của các kí tự PAD do chúng ta thêm vào
    """
    src_mask = (src != 0).unsqueeze(-2)

    if trg is not None:
        trg_mask = (trg != 0).unsqueeze(-2)
        size = trg.size(1) # get seq_len for matrix
        np_mask = nopeak_mask(size)
        if trg.is_cuda:
            np_mask.cuda()
        trg_mask = trg_mask & np_mask

    else:
        trg_mask = None
    return src_mask, trg_mask

in_mask, out_mask = create_masks(T_demand, Used_path)
in_mask.shape

torch.Size([1, 25, 1, 25])

### Extract OD demand and path set (X and Y)
X: OD demand, graph (link feature), path, link-path adj \
Y: path flow

In [ ]:
%run helpers.py
%run attention.py

class Dataset():
    def __init__(self, size, input_dim0, input_dim1, output_dim0, output_dim1, start_from=0):
        super().__init__()
        self.path_encoded = path_encoder() # Get path encode dictionary
        self.entries = size
        self.X = torch.zeros([size, input_dim0, input_dim1], dtype=torch.float32)
        self.Y = torch.zeros([size, output_dim0, output_dim1], dtype=torch.float32)

        for i in tqdm(range(size)) :
            file_name = f"Output/5by5_Data{start_from+i}"
            x, y = generate_xy(file_name, self.path_encoded)
            self.X[i] = x
            self.Y[i] = y
    
    def __len__(self):
        return self.entries

    def __getitem__(self, idx):
        data_point = self.X[idx]
        data_label = self.Y[idx]
        return data_point, data_label
    

In [ ]:
# %run helpers.py

# Load first 1000 sample data
train_size = 1000
train_dataset = Dataset(train_size, 625, 1165, 625, 3)
# this code create batch data, shape batch_size x seq_length x dim
train_data_loader = data.DataLoader(train_dataset, batch_size=128, shuffle=False, drop_last=False)

  0%|          | 0/1000 [00:00<?, ?it/s]

### Test Encoder

In [ ]:
input = torch.zeros(128, 625, 1165).float()
for i, output in train_data_loader:
    print(i.shape)
    if i.shape[0] == 128: 
        input = i
    # print("Output shape: ", output.shape)

NameError: name 'torch' is not defined

In [ ]:

mask_tensor = torch.rand(input.shape[0], 1, input.shape[-2]).float() #shape 4 x 1 x 625
x2 = Encoder(input_dim=input.shape[-1], d_model=512, N=6, heads=8, dropout=0.1)(input, mask_tensor)
print("Encoder output shape: ", x2.shape)

### Test Decoder